In [1]:
import os, time, copy
import numpy as np
import torch
import torchvision as tv
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
from utils import train_model

In [2]:
# Get data 2006-2014 from the following link: https://darchive.mblwhoilibrary.org/handle/1912/7341
# Unzip it in the following directory
data_dir = './data/2013'

In [3]:
# Pct Val
pct_val = 0.2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True
input_size = 224


train_transform = tv.transforms.Compose([
        tv.transforms.RandomResizedCrop(input_size),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = tv.transforms.Compose([
        tv.transforms.Resize(input_size),
        tv.transforms.CenterCrop(input_size),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
dataset = tv.datasets.ImageFolder(data_dir, train_transform)
num_train = int((1-pct_val)*len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_train, len(dataset)-num_train])
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

# Initialize dataloaders
num_classes = len(dataset.classes)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Initializing Datasets and Dataloaders...
Using device cpu


In [4]:
# Set up model
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

model_ft = tv.models.resnet18(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.fc.in_features
model_ft.fc = torch.nn.Linear(num_ftrs, num_classes)

In [5]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.AdamW(params_to_update, lr=1e-4)

Params to learn:
	 fc.weight
	 fc.bias


In [6]:
# Setup the loss fxn
criterion = torch.nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, {'train': train_dataloader, 'val': val_dataloader}, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/14
----------


  0%|▍                                                                                                                                                                                                                                       | 79/42124 [00:18<2:41:16,  4.35it/s]


KeyboardInterrupt: 